## 4-1. シュレディンガー方程式とは、量子ダイナミクスとは

### ダイナミクスのシミュレーションとは
古典・量子によらず、コンピュータの応用先として重要なものに、「**物理系のダイナミクスのシミュレーション**」がある。

物理系のシミュレーションは、身の回りの生活を成り立たせる上で無くてはならない技術である。例えば、ビルや航空機の設計、気象予報、創薬といった、ありとあらゆる場面でコンピュータを用いた物理系のシミュレーションが行われている。物理系のシミュレーションのゴールは、系（着目している対象）の**初期状態**（最初の状態）と、その系の**ダイナミクス＝時間発展**（系が時間の経過に伴ってどう変化するか）のルールが与えられたときに、一定時間後に系がどういう状態にあるかを知ることである。

例えばビリヤードのシミュレーションを考えよう。初期状態は各玉の最初の位置と速度である。また、古典力学系なので時間発展のルールはニュートンの運動方程式（微分方程式）である。これらを与えてやることで、ブレイクショットの終了時に各玉がどこにあるかを予測することができるのである。

上の例のように、系の時間発展のルールは、一般に**微分方程式**で表される。古典力学系ならニュートンの運動方程式、電磁気学系ならマクスウェル方程式、流体力学系ならナビエ・ストークス方程式、といった具合である。したがって、物理系のシミュレーションの本質は、微分方程式を数値的に解くことに帰着される。そして、当然ながら量子系のシミュレーション、すなわち量子力学に従う系のシミュレーションもその中に含まれる。

### 量子系のシミュレーション
量子系はシュレディンガー方程式（後述）というものに従って時間発展する。よって、量子系のダイナミクスをシミュレートするにはシュレディンガー方程式を数値的に解くことが必要となる。他の物理系のシミュレーションと同様、古典コンピュータを用いて量子系のシミュレーションを行うことは原理的には可能である。実際、様々な近似を巧みに用いることで、古典コンピュータを用いた量子系のシミュレーションは一定の成功を収めてきた。

しかし、**量子系を古典コンピュータでシミュレーションする場合、系のサイズ（粒子やスピンの数）が大きくなると、計算にかかる時間が指数関数的に増大するため、すぐに太刀打ちできなくなってしまう**。例えば、$n$個の量子ビットのある系をシミュレーションするためにはおよそ$2^n$個の方程式を解かなければならない（参照：Nielsen-Chuang `4.7 Simulation of quantum systems`）。

そこで、**指数関数的な速度向上が期待される量子コンピュータを用いて量子系のシミュレーションを行う**ことで、上記の問題を解決しようというアイデアが生まれた。まさに「量子の問題は量子で解決」である。

本章では、量子コンピュータを用いた量子系のダイナミクスのシミュレーション手法の内、代表的なものについて学んでいく。

### 用語の整理
ここで、量子ダイナミクスのシミュレーションに出てくる用語を整理しておこう。

#### シュレディンガー方程式

$$
i\hbar \frac{\partial |\psi(t)\rangle}{\partial t} = H |\psi(t)\rangle
$$

量子力学の基礎方程式であり、ミクロな世界を記述するルールである。$|\psi(t)\rangle$は量子状態、$H$は以下で説明するハミルトニアンという演算子で、$\hbar$はプランク定数と呼ばれる定数である（理論研究では、$\hbar=1$とする単位系を用いることが多い）。ミクロな系は、このシュレディンガー方程式に則って時間発展する。古典力学におけるニュートンの運動方程式に相当する。   
特に、状態が時間に依存しない場合、以下の式をシュレディンガー方程式と呼ぶことも多い。

$$
H |\psi\rangle = E|\psi\rangle  \:\:\: \text{(系が時間に依存しない場合)}
$$

ここで$E$は系の（固有）エネルギーと呼ばれる。

#### ハミルトニアン
シュレディンガー方程式に出てくる演算子であり、系の時間発展を司る。量子コンピュータでシミュレーションする範囲においては、ただの大きな行列だと思っておいても良い。第1章で学んだ量子操作と異なる点は、ハミルトニアンはエルミート$H=H^\dagger$だということである。そのため、一般にハミルトニアンを作用させた状態$H|\psi\rangle$を量子コンピュータ上で作ることはできない。（ユニタリかつエルミートであるパウリ演算子$X,Y,Z$という例外を除く）

ハミルトニアンは、**系のエネルギーに対応するオブザーバブル**(次項)でもある。ハミルトニアンには、粒子同士に働く力や、外部からの力（電場・磁場など）といった系の情報のすべてが詰まっている。このハミルトニアンをシュレディンガー方程式に入れ、それを解くことで、系のエネルギーの値が得られる。

#### オブザーバブル（物理量）
量子力学では、物理量（エネルギー、磁化＝系の示す磁力の大きさ、など）は状態に作用する**エルミート演算子**として記述される。演算子$A$がエルミートであるとは、$A=A^\dagger$を満たすことである（第1章で学んだ量子操作を表すユニタリ演算子$UU^\dagger=I$と混同しないように注意）。   
$A$を対角化した時の固有値を$\{a_i \}_i$、固有ベクトルを$\{|a_i\rangle \}_i$とする。状態$|\psi\rangle$について、物理量$A$を観測（射影測定）した場合、観測値$a_i$が

$$
 |\psi\rangle = \sum_i c_i |a_i\rangle
$$

の係数$|c_i|^2$に比例した確率で得られる。実は第1章で学んだ射影測定は、$Z$というオブザーバブルに関する射影測定のことだったのである。

#### エネルギー固有状態（基底状態・励起状態）
前述のように、ハミルトニアン$H$は系のエネルギーを表すオブザーバブルだから、その固有値が分かると系が取り得るエネルギーの値（固有エネルギー）が分かる。系のエネルギーが分かると何が嬉しいのだろうか？それは、「実際にどの状態が実現しやすいか」が分かるという点である。

与えられたハミルトニアンのもとでシュレディンガー方程式を解くと、一般に複数の解（状態とエネルギーの組）が得られる。それらの中で、最も低いエネルギーを持つ、すなわち最も安定な状態は「**基底状態**」と呼ばれ、自然の中で最も実現しやすい状態である（系の温度が上がると、より高エネルギーな状態も出現しやすくなる）。

物質科学や量子化学の文脈においては、各状態のエネルギーを知ることは非常に重要である。なぜなら、どんな結晶構造が安定なのか・どんな化学反応が実際に起こるのか・反応に必要なエネルギーはどれくらいか、と言ったことを検討することが可能になるからである。

#### ダイナミクス（時間発展）
ハミルトニアン、つまり系が時間に依存しない場合のシュレーディンガー方程式を形式的にとくと

$$
|\psi(t)\rangle = e^{-iH/\hbar t} |\psi(0)\rangle
$$

となる。量子ダイナミクスのシミュレーションとは、つまるところ、この$e^{-iH/\hbar t} |\psi(0)\rangle$をいかに計算するかという問題なのである。（ハミルトニアン$H$は量子ビットの数$n$に対して$2^n$次元となり指数的に大きいので、古典コンピュータでの計算は非常に大変である）   
よくある問題設定は、とある初期状態$|\psi(0)\rangle$からスタートし、時間$t$だけ経った後の何らかの物理量$A$の期待値$A(t) =\langle\psi(0)|e^{iH/\hbar t} A e^{-iH/\hbar t} |\psi(0)\rangle$を計算する、というものである。

※進んだ註：Dojoでは扱っていないが、混合状態と呼ばれる状態についてのダイナミクスは、[von-Neumann方程式](https://ja.wikipedia.org/wiki/密度行列#時間発展)と呼ばれるシュレディンガー方程式に似た方程式で記述される。